In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

Using TensorFlow backend.


In [2]:
df = pd.read_excel('name_gender.xlsx')
df = df.drop("probability",axis = 1)
print(df.shape)
df.head()

(95025, 2)


,name,gender
0,Aaban,M
1,Aabha,F
2,Aabid,M
3,Aabriella,F
4,Aada,F


In [3]:
df.isna().any()

name      False
gender    False
dtype: bool

In [4]:
df = df.drop(95024, axis = 0)
names = df['name'].values
gender = df["gender"].values
print(names.shape)

(95024,)


In [5]:
MAX_LEN=np.max([len(s) for s in names])
print(MAX_LEN)

15


In [6]:
vocab = {'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [7]:
def convertNameToVec(name,vocab):
  vector=np.zeros((MAX_LEN,27))
  for i in range(min(MAX_LEN,len(name))):
    vec = np.zeros(27)
    vec[vocab[name[i]]] = 1
    vector[i,:] = vec
  return vector

In [8]:
convertNameToVec("Ghulam".lower(),vocab)

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],


In [9]:
names=np.array([convertNameToVec(x.lower(),vocab) for x in names])
print(names.shape)

(95024, 15, 27)


In [10]:
unique, counts = np.unique(gender, return_counts=True)
gender_count = dict(zip(unique, counts))
print(gender_count)

gender = 1*(gender == "M")

unique, counts = np.unique(gender, return_counts=True)
gender_count = dict(zip(unique, counts))
print(gender_count)

{'F': 60303, 'M': 34721}
{0: 60303, 1: 34721}


In [11]:
gender.shape

(95024,)

In [12]:
X_train, X_test ,y_train ,y_test =train_test_split(names,gender,test_size=0.2,random_state=0)

In [13]:
def LSTM_Model():
    model=Sequential()
    model.add(LSTM(27,input_shape= (15,27)))
    model.add(Dense(14, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
    return model

In [15]:
lstm_model = LSTM_Model()
lstm_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 27)                5940      
_________________________________________________________________
dense_3 (Dense)              (None, 14)                392       
_________________________________________________________________
dropout_2 (Dropout)          (None, 14)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 15        
Total params: 6,347
Trainable params: 6,347
Non-trainable params: 0
_________________________________________________________________


In [16]:
lstm_model.fit(X_train, y_train, epochs=100, batch_size=200)

Epoch 1/100
76019/76019 [==============================] - 7s 98us/step - loss: 0.5516 - accuracy: 0.7327 0s - l
Epoch 2/100
76019/76019 [==============================] - 7s 94us/step - loss: 0.4726 - accuracy: 0.7933
Epoch 3/100
76019/76019 [==============================] - 7s 96us/step - loss: 0.4390 - accuracy: 0.8124
Epoch 4/100
76019/76019 [==============================] - 7s 89us/step - loss: 0.4143 - accuracy: 0.8243
Epoch 5/100
76019/76019 [==============================] - 7s 92us/step - loss: 0.4010 - accuracy: 0.8311
Epoch 6/100
76019/76019 [==============================] - 7s 94us/step - loss: 0.3884 - accuracy: 0.8363
Epoch 7/100
76019/76019 [==============================] - 7s 91us/step - loss: 0.3809 - accuracy: 0.8412
Epoch 8/100
76019/76019 [==============================] - 7s 92us/step - loss: 0.3730 - accuracy: 0.8453
Epoch 9/100
76019/76019 [==============================] - 7s 93us/step - loss: 0.3646 - accuracy: 0.8479
Epoch 10/100
76019/76019 [=============

76019/76019 [==============================] - 7s 95us/step - loss: 0.2572 - accuracy: 0.8979
Epoch 78/100
76019/76019 [==============================] - 7s 94us/step - loss: 0.2547 - accuracy: 0.8986
Epoch 79/100
76019/76019 [==============================] - 7s 96us/step - loss: 0.2563 - accuracy: 0.8983
Epoch 80/100
76019/76019 [==============================] - 7s 95us/step - loss: 0.2555 - accuracy: 0.8993
Epoch 81/100
76019/76019 [==============================] - 7s 96us/step - loss: 0.2545 - accuracy: 0.8995
Epoch 82/100
76019/76019 [==============================] - 7s 94us/step - loss: 0.2545 - accuracy: 0.8993
Epoch 83/100
76019/76019 [==============================] - 7s 94us/step - loss: 0.2539 - accuracy: 0.8999
Epoch 84/100
76019/76019 [==============================] - 7s 90us/step - loss: 0.2541 - accuracy: 0.8993
Epoch 85/100
76019/76019 [==============================] - 7s 93us/step - loss: 0.2531 - accuracy: 0.9006
Epoch 86/100
76019/76019 [========================

In [17]:
_, accuracy = lstm_model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

19005/19005 [==============================] - 2s 128us/step
Accuracy: 89.18


In [19]:
y_test_pred = lstm_model.predict(X_test)
y_test_pred = [int(round(x[0],0)) for x in y_test_pred]

unique, counts = np.unique(y_test_pred, return_counts=True)
pred = dict(zip(unique, counts))
print("Predicted test values:\n", pred)

unique, counts = np.unique(y_test, return_counts=True)
pred = dict(zip(unique, counts))
print("Original test values:\n", pred)

Predicted test values:
 {0: 12302, 1: 6703}
Original test values:
 {0: 12001, 1: 7004}


In [20]:
cm = confusion_matrix(y_test, y_test_pred)
print(cm)
print('Testing Accuracy = {}%'.format(  (cm[0][0] + cm[1][1]) / (cm[0][0] + cm[1][1] + cm[1][0] + cm[0][1])))

[[11123   878]
 [ 1179  5825]]
Testing Accuracy = 0.8917653249144962%


In [40]:
def predictGender(name):
    customNameVector=convertNameToVec(name.lower(),vocab)
    # making input 3 dimensinal
    customNameVector=customNameVector[np.newaxis,:,:]
    pred = lstm_model.predict_proba(customNameVector)
    if(pred[0]>0.5):
        print('Male\nwith {}% probability'.format(round(pred[0][0]*100,2)))
    else:
        print('Female\nwith {}% probability'.format(round((1-pred[0][0])*100,2)))

In [29]:
predictGender("Ghulam")

Male
with 91.79% probability


In [30]:
predictGender("Muhammad")

Male
with 99.38% probability


In [41]:
predictGender("Abagail")

Female
with 95.38% probability


In [42]:
predictGender("Hosea")

Male
with 73.15% probability


In [45]:
predictGender("Shumail")

Male
with 84.48% probability


In [46]:
predictGender("Shumaila")

Female
with 99.74% probability


In [47]:
predictGender("Alex")

Male
with 66.65% probability
